## Рекомендательные системы

## Инсталляция 

In [49]:
#!pip install implicit

In [50]:
#!pip install lightgbm

In [51]:
#!pip install catboost

## Библиотеки

In [98]:
import pandas as pd
import numpy as np


def prefilter_items(data, take_n_popular=10000, item_features=None):
    # Уберем не интересные для рекоммендаций категории (department)
    if item_features is not None:
        department_size = pd.DataFrame(item_features. \
                                       groupby('department')['item_id'].nunique(). \
                                       sort_values(ascending=False)).reset_index()

        department_size.columns = ['department', 'n_items']
        rare_departments = department_size[department_size['n_items'] < 150].department.tolist()
        items_in_rare_departments = item_features[
            item_features['department'].isin(rare_departments)].item_id.unique().tolist()

        data = data[~data['item_id'].isin(items_in_rare_departments)]

    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    data = data[data['price'] > 2]

    # Уберем слишком дорогие товары
    data = data[data['price'] < 50]

    # Возбмем топ по популярности
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

    top = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()

    # Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
    data.loc[~data['item_id'].isin(top), 'item_id'] = 999999


    return data

def postfilter_items(user_id, recommednations):
    pass

In [99]:
class MainRecommender:
    """Рекомендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, weighting=True, fake_id=999999):
        self.weighting = weighting
        self.fake_id = fake_id

        # Топ покупок каждого пользователя
        self.top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.top_purchases.sort_values('quantity', ascending=False, inplace=True)
        if fake_id is not None:
            self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != fake_id]

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values('quantity', ascending=False, inplace=True)
        if fake_id is not None:
            self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != fake_id]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()

        # Подготовка user-item матриц
        self.user_item_matrix = self._prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = self._prepare_dicts(
            self.user_item_matrix)
        self.user_item_matrix_for_pred = self.user_item_matrix
        if weighting == 'bm25':
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T.tocsr()
        elif weighting == 'tfidf':
            self.user_item_matrix = tfidf_weight(self.user_item_matrix.T).T.tocsr()

        # Обучение моделей
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def _prepare_matrix(data):
        """Готовит user-item матрицу"""

        user_item_matrix = pd.pivot_table(data,
                                          index='user_id',
                                          columns='item_id',
                                          values='quantity',  # Можно пробовать другие варианты sales_value
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit

        return user_item_matrix

    @staticmethod
    def _prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=10, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).tocsr())

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=30, regularization=0.01, iterations=20, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)

        model.fit(csr_matrix(user_item_matrix).tocsr())

        return model

    def _update_dict(self, user_id):
        """Если появился новый user, то нужно обновить словари"""

        if user_id not in self.userid_to_id.keys():
            max_id = max(list(self.userid_to_id.values()))
            max_id += 1

            self.userid_to_id.update({user_id: max_id})
            self.id_to_userid.update({max_id: user_id})

    def _get_similar_item(self, item_id):
        """Находит товар, похожий на item_id"""

        recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)  # Товар похож на себя -> рекомендуем 2 товара
        top_rec = recs[0][1]  # Берем 2-ой (не товар)
        return self.id_to_itemid[top_rec] # возвращаем индексы

    def _extend_with_top_popular(self, recommendations, N=5):
        """Если кол-во рекоммендаций < N, то дополняем их топ-популярными"""

        if len(recommendations) < N:
            top_popular = [rec for rec in self.overall_top_purchases[:N] if rec not in recommendations]
            recommendations.extend(top_popular)
            recommendations = recommendations[:N]

        return recommendations

    def get_recommendations(self, user, model, N=5):
        """Рекомендации через стандартные библиотеки implicit"""

        sparse_user_item = csr_matrix(self.user_item_matrix_for_pred).tocsr()
        self._update_dict(user_id=user)
        filter_items = [] if self.fake_id is not None else [self.itemid_to_id[self.fake_id]]
        res = model.recommend(userid=self.userid_to_id[user],
                              user_items=sparse_user_item[self.userid_to_id[user]],
                              N=N,
                              filter_already_liked_items=False,
                              filter_items=filter_items,
                              recalculate_user=True)
        mask = res[1].argsort()[::-1]
        res = [self.id_to_itemid[rec] for rec in res[0][mask]]
        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def _get_recommendations(self, user, model, N=5):
        """Рекомендации через стандартные библиотеки implicit"""

        self._update_dict(user_id=user)
        filter_items = [] if self.fake_id is not None else [self.itemid_to_id[self.fake_id]]
        res = model.recommend(userid=self.userid_to_id[user],
                              user_items=csr_matrix(self.user_item_matrix_for_pred).tocsr(),
                              N=N,
                              filter_already_liked_items=False,
                              filter_items=filter_items,
                              recalculate_user=True)
        mask = res[1].argsort()[::-1]
        res = [self.id_to_itemid[rec] for rec in res[0][mask]]
        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_als_recommendations(self, user, n=5):
        """Рекомендации через стандартные библиотеки implicit"""

        self._update_dict(user_id=user)
        return self.get_recommendations(user, model=self.model, N=n)

    def get_own_recommendations(self, user, n=5):
        """Рекомендуем товары среди тех, которые пользвотель уже купил (ItemItemRecommender)"""

        self._update_dict(user_id=user)
        return self._get_recommendations(user, model=self.own_recommender, N=n)

    def get_similar_users_recommendation(self, user, n=5):
        """Рекомендуем топ-N товаров, среди купленных похожими пользователями"""

        res = []
        # Находим похожих юзеров
        similar_users = self.model.similar_users(self.userid_to_id[user], N=n+1)
        similar_users = [rec for rec in similar_users[0]][1:]
        # Находим купленные топ товары, купленные этими юзерами
        for user in similar_users:
            # user_rec = self._get_recommendations(user, model=self.own_recommender, N=1)
            user_rec = self._get_recommendations(user, model=self.own_recommender, N=n)
            # выберем одну рекомендацию из списка рекомендаций юзера
            for r in user_rec:
                if r not in res:
                    res.append(r)
                    break
        res = np.array(res).flatten()

        assert len(res) == n, 'Количество рекомендаций != {}'.format(n)
        return res

    def get_similar_items_recommendation(self, user, n=5):
        """Рекомендуем товары, похожие на топ-N купленных пользователем товаров"""

        # выясняем топ-N купленных юзером товаров
        top_users_purchases = self.user_item_matrix.iloc[self.userid_to_id[user]].sort_values(ascending=False).head(
            n).reset_index()
        # находим по одной рекомендации для каждого топ-N товара.
        res = top_users_purchases['item_id'].apply(lambda x: self._get_similar_item(x)).tolist()
        res = np.array(res)

        assert len(res) == n, 'Количество рекомендаций != {}'.format(n)
        return res

In [100]:
"""
Metrics

"""
import numpy as np

def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

def hit_rate_at_k(recommended_list, bought_list, k=5):
    return hit_rate(recommended_list[:k], bought_list)

def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    flags = np.isin(recommended_list, bought_list)
    return np.dot(flags, prices_recommended).sum() / prices_recommended.sum()

def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)


def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    flags = np.isin(recommended_list, bought_list)
    return np.dot(flags, prices_recommended).sum() / prices_bought.sum()


def ap_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[recommended_list <= k]

    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    amount_relevant = len(relevant_indexes)


    sum_ = sum(
        [precision_at_k(recommended_list, bought_list, k=index_relevant + 1) for index_relevant in relevant_indexes])
    return sum_ / amount_relevant



## Решение

In [101]:
import pandas as pd
import numpy as np
import pickle

from scipy.sparse import csr_matrix
from implicit import als

import lightgbm as lgb
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# from src.metrics import precision_at_k, recall_at_k
# from src.utils import prefilter_items
# from src.recommenders import MainRecommender

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [102]:
# from google.colab import drive
# drive.mount('/content/drive')

### Загрузка и обработка данных

In [103]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [104]:
data.shape

(2396804, 12)

In [105]:
test = pd.read_csv('retail_test.csv') #датасет из baseline https://github.com/geangohn/recsys-tutorial
test.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0


In [106]:
test.shape

(88734, 12)

In [107]:
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [108]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [109]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


#### Схема обучения и валидации:
-- давние покупки -- | -- 6 недель -- | -- 3 недель -- 

In [110]:
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

In [111]:
users_lvl_1 = data_train_lvl_1.user_id.unique()
users_lvl_2 = data_val_lvl_1.user_id.unique()
users_lvl_3 = data_val_lvl_2.user_id.unique()

new_users_lvl_2 = list(set(users_lvl_2) - set(users_lvl_1))
new_users_lvl_3 = list(set(users_lvl_3) - (set(users_lvl_1) | set(users_lvl_2)))

add_to_lvl_2 = list(set(users_lvl_3) - (set(users_lvl_2)))

new_users_lvl_2, new_users_lvl_3, len(add_to_lvl_2)

([1984], [], 126)

In [112]:
users_lvl_1.shape, users_lvl_2.shape, users_lvl_3.shape, 

((2498,), (2154,), (2042,))

In [113]:
# Уменьшение размерности
n_items_before = data_train_lvl_1['item_id'].nunique()
data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=10000)
n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 10001


### Генерация  и отбор новых признаков для модели второго уровня

In [114]:
user_features['age_desc'].replace(
    {'19-24': 22, '25-34': 30, '35-44': 40, '45-54': 50, '55-64': 60, '65+': 70},
    inplace=True)

user_features['marital_status_code'].replace(
    {'U': 0, 'A': 1, 'B': 2}, inplace=True)

user_features['income_desc'].replace(
    {'Under 15K': 10, '15-24K': 20, '25-34K':30, '35-49K': 40,
     '50-74K': 62, '75-99K': 87, '100-124K': 112, '125-149K': 137, 
     '150-174K': 162, '175-199K': 187, '200-249K': 225, '250K+':275}, inplace=True)

user_features['homeowner_desc'].replace(
    {'Unknown': 0, 'Probable Renter': 1, 'Renter': 2,
     'Probable Owner': 3, 'Homeowner': 4}, inplace=True)

user_features['hh_comp_desc'].replace(
    {'Unknown': 0, 'Single Male': 1, 'Single Female': 2,
     '1 Adult Kids': 3, '2 Adults No Kids': 4, '2 Adults Kids':5},inplace=True)

user_features['household_size_desc'].replace({'5+': 5}, inplace=True) 

user_features['kid_category_desc'].replace(
    {'None/Unknown': 0, '3+': 3}, inplace=True)

user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,70,1,40,4,4,2,0,1
1,50,1,62,4,4,2,0,7


In [115]:
names = ['manufacturer', 'department', 'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product']
for name in names:
    new_name = name + '_freq'
    a = item_features[name].value_counts()
    ind = a.index.tolist()
    for i in ind:
        item_features.loc[item_features[name] == i, new_name] = a[i]

item_features['brand'] = np.where(item_features['brand']=='Private', 0, 1)

commodities = item_features.commodity_desc.value_counts()
commodities_list = commodities.keys().tolist()
for i, name in enumerate(commodities_list):
    item_features.loc[item_features['commodity_desc'] == name, 'commodity_category'] = i

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,manufacturer_freq,department_freq,commodity_desc_freq,sub_commodity_desc_freq,curr_size_of_product_freq,commodity_category
0,25671,2,GROCERY,1,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,1411.0,39021.0,29.0,29.0,12.0,270.0
1,26081,2,MISC. TRANS.,1,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,1411.0,490.0,490.0,429.0,30607.0,56.0


In [116]:
def get_new_features(data_train_lvl_1, n_factors=20):
    
    # час совершения транзакции
    data = data_train_lvl_1.copy()
    data['hour'] = data['trans_time'] // 100
    user_item_features = data.groupby(['user_id', 'item_id'])['hour'].median().reset_index()
    user_item_features.columns = ['user_id', 'item_id', 'median_sales_hour']
    
    # день недели совершения транзакции
    data['weekday'] = data['day'] % 7
    df = data.groupby(['user_id', 'item_id'])['weekday'].median().reset_index()
    df.columns = ['user_id', 'item_id', 'median_weekday']
    user_item_features = user_item_features.merge(df, on=['user_id', 'item_id'])
    
    # cреднее кол-во дней между покупками
    df = data.groupby('user_id')['day'].nunique().reset_index()
    df['mean_visits_interval'] = (data.groupby('user_id')['day'].max() - data.groupby('user_id')['day'].min()) / df['day']
    user_item_features = user_item_features.merge(df[['user_id', 'mean_visits_interval']], on=['user_id'])
    
    # средний чек корзины клиента
    df = data.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()
    df = df.groupby('user_id')['sales_value'].mean().reset_index()
    df.columns = ['user_id', 'mean_check']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # кол-во магазинов, в которых продавался товар
    df = data.groupby(['item_id'])['store_id'].nunique().reset_index()
    df.columns = ['item_id', 'n_stores']
    user_item_features = user_item_features.merge(df, on=['item_id'])
    
    # кол-во уникальных товаров, купленных клиентом
    df = data.groupby(['user_id'])['item_id'].nunique().reset_index()
    df.columns = ['user_id', 'n_items']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # кол-во транзакций клиента
    df = data.groupby(['user_id'])['item_id'].count().reset_index()
    df.columns = ['user_id', 'n_transactions']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # mean / max / std кол-ва уникальных товаров в корзине клиента
    df = data.groupby(['user_id', 'basket_id'])['item_id'].nunique().reset_index()
    df1 = df.groupby('user_id')['item_id'].mean().reset_index()
    df1.columns = ['user_id', 'mean_n_items_basket']
    user_item_features = user_item_features.merge(df1, on=['user_id'])

    df2 = df.groupby('user_id')['item_id'].max().reset_index()
    df2.columns = ['user_id', 'max_n_items_basket']
    user_item_features = user_item_features.merge(df2, on=['user_id'])

    df3 = df.groupby('user_id')['item_id'].std().reset_index()
    df3.columns = ['user_id', 'std_n_items_basket']
    user_item_features = user_item_features.merge(df3, on=['user_id'])
    
    # mean / max / std кол-ва уникальных категорий в корзине клиента
    data = data.merge(item_features[['item_id', 'commodity_desc']], on=['item_id'])
    df = data.groupby(['user_id', 'basket_id'])['commodity_desc'].nunique().reset_index()
    df1 = df.groupby('user_id')['commodity_desc'].mean().reset_index()
    df1.columns = ['user_id', 'mean_n_item_categories_basket']
    user_item_features = user_item_features.merge(df1, on=['user_id'])

    df2 = df.groupby('user_id')['commodity_desc'].max().reset_index()
    df2.columns = ['user_id', 'max_n_item_categories_basket']
    user_item_features = user_item_features.merge(df2, on=['user_id'])

    df3 = df.groupby('user_id')['commodity_desc'].std().reset_index()
    df3.columns = ['user_id', 'std_n_item_categories_basket']
    user_item_features = user_item_features.merge(df3, on=['user_id'])
    return user_item_features

In [117]:
user_item_features = get_new_features(data_train_lvl_1)
user_item_features.head(2)

,user_id,item_id,median_sales_hour,median_weekday,mean_visits_interval,mean_check,n_stores,n_items,n_transactions,mean_n_items_basket,max_n_items_basket,std_n_items_basket,mean_n_item_categories_basket,max_n_item_categories_basket,std_n_item_categories_basket
0,1,823721,13.0,4.0,NaN,37.4205,94,236,661,10.116667,23,6.095543,8.803571,18,4.602053
1,1,823990,15.0,6.0,NaN,37.4205,95,236,661,10.116667,23,6.095543,8.803571,18,4.602053


In [118]:
user_item_features.shape

(401048, 15)

In [119]:
def get_candidates(data_train_lvl_1, data_train_lvl_2, N, add_to_lvl_2):
    recommender = MainRecommender(data_train_lvl_1) # weighting='tfidf'

    users_lvl_1 = data_train_lvl_1['user_id'].unique()
    users_lvl_2 = data_train_lvl_2['user_id'].unique().tolist()
    if add_to_lvl_2:
        users_lvl_2 += add_to_lvl_2

    current_users = list(set(users_lvl_2) & set(users_lvl_1))    
    new_users = list(set(users_lvl_2) - set(users_lvl_1))

    df = pd.DataFrame(users_lvl_2, columns=['user_id'])
    cond_1 = df['user_id'].isin(current_users)
    df.loc[cond_1, 'candidates'] = df.loc[cond_1, 'user_id'].apply(
        lambda x: recommender.get_own_recommendations(x, N))

    if new_users:
        cond_2 = df['user_id'].isin(new_users)
        df.loc[cond_2, 'candidates'] = df.loc[cond_2, 'user_id'].apply(
            lambda x: recommender.overall_top_purchases[:N])
        
    return df

In [120]:
def get_targets_lvl_2(data_train_lvl_1, data_train_lvl_2, user_item_features, N, add_to_lvl_2=None):
    
    users_lvl_2 = get_candidates(data_train_lvl_1, data_train_lvl_2, N, add_to_lvl_2)
    
    df = pd.DataFrame({'user_id': users_lvl_2['user_id'].values.repeat(N),
                       'item_id': np.concatenate(users_lvl_2['candidates'].values)})

    targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
    targets_lvl_2['target'] = 1 

    targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')
    targets_lvl_2['target'].fillna(0, inplace= True)
    
    targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(
        user_item_features, on=['user_id', 'item_id'], how='left')
    
    return targets_lvl_2

In [121]:
%%time
N = 500
targets_lvl_2 = get_targets_lvl_2(data_train_lvl_1, data_train_lvl_2, user_item_features, N, add_to_lvl_2)

print(f'Число пользователей: {targets_lvl_2.user_id.nunique()}')
print(f'Среднее число покупок: {round(targets_lvl_2["target"].mean(), 4)}')

targets_lvl_2.head(2)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/10001 [00:00<?, ?it/s]

Число пользователей: 2280
Среднее число покупок: 0.0159
CPU times: total: 13min 16s
Wall time: 13min 12s


,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,manufacturer_freq,...,mean_check,n_stores,n_items,n_transactions,mean_n_items_basket,max_n_items_basket,std_n_items_basket,mean_n_item_categories_basket,max_n_item_categories_basket,std_n_item_categories_basket
0,2070,999999,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.48875,488.0,346.0,592.0,4.234375,30.0,6.175242,3.5,25.0,4.878524
1,2070,1029743,0.0,69.0,GROCERY,0.0,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA,12676.0,...,23.48875,112.0,346.0,592.0,4.234375,30.0,6.175242,3.5,25.0,4.878524


In [122]:
SELECTED_FEATURES_NAMES = targets_lvl_2.columns.tolist()
to_drop=['user_id', 'item_id', 'target', 'manufacturer', 'department']
# categorical_features = targets_lvl_2.select_dtypes(include=[np.object])columns.tolist()
categorical_features = ['marital_status_code', 'homeowner_desc',
                        'hh_comp_desc', 'manufacturer', 'commodity_category',
                        'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product',
                        'household_size_desc', 'kid_category_desc']
# targets_lvl_2[categorical_features] = targets_lvl_2[categorical_features].astype(str)

SELECTED_FEATURES_NAMES = [_ for _ in SELECTED_FEATURES_NAMES if not _ in (categorical_features+to_drop)]

### Модели

In [123]:
X_train, X_valid, y_train, y_valid = train_test_split(targets_lvl_2[SELECTED_FEATURES_NAMES].fillna(0),
                                                          targets_lvl_2[['target']],
                                                          test_size=0.2, random_state=27,
                                                          stratify=targets_lvl_2[['target']])

#### CatBoost

In [124]:
%%time

dtrain = Pool(data=X_train, label=y_train)
dvalid = Pool(data=X_valid, label=y_valid)
params_cb = {"n_estimators":10000,
             "learning_rate": 0.1,
             "loss_function": "Logloss",
             "eval_metric": "AUC",
             "task_type": "CPU",
             "max_bin": 30,
             "early_stopping_rounds": 100,
             "verbose": 500,
             "max_depth": 10,
             "l2_leaf_reg": 80,
             "thread_count": 6,
             "random_seed": 21} 


model_cb = CatBoostClassifier(**params_cb)
model_cb.fit(dtrain,
             eval_set=[dvalid]
            )



0:	test: 0.8413332	best: 0.8413332 (0)	total: 302ms	remaining: 50m 18s
500:	test: 0.9290083	best: 0.9290083 (500)	total: 2m 30s	remaining: 47m 33s
1000:	test: 0.9331396	best: 0.9331402 (999)	total: 5m 1s	remaining: 45m 9s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9343594753
bestIteration = 1347

Shrink model to first 1348 iterations.
CPU times: total: 29min 10s
Wall time: 7min 11s


#### LightGBM

In [125]:
%%time

dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

params_lgb = {"boosting_type": "gbdt",
              "objective": "binary",
              "metric": "auc",
              "num_boost_round": 10000,
              "learning_rate": 0.1,
              #"class_weight": 'balanced',
              "max_depth": 10,
              "n_estimators": 10000,
              "n_jobs": 6,
              "num_leaves": 31,
              "seed": 87} 


model_lgb = lgb.train(params=params_lgb,
                      train_set=dtrain,
                      valid_sets=[dtrain, dvalid],
                      verbose_eval=500,
                      early_stopping_rounds=100)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 14609, number of negative: 901836
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.033247 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2815
[LightGBM] [Info] Number of data points in the train set: 916445, number of used features: 21
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.015941 -> initscore=-4.122795
[LightGBM] [Info] Start training from score -4.122795
Training until validation scores don't improve 

In [126]:
predictions_cb_train = model_cb.predict_proba(targets_lvl_2[SELECTED_FEATURES_NAMES].fillna(0))[:, 1]
predictions_lgb_train = model_lgb.predict(targets_lvl_2[SELECTED_FEATURES_NAMES].fillna(0))

In [127]:
preds_train_2_mods = 0.6*predictions_lgb_train + 0.4* predictions_cb_train
roc_auc_score(targets_lvl_2['target'], preds_train_2_mods)

0.9499918129133529

**Результат:**

**Изменения values в _prepare_matrix, take_n_popular в prefilter_items и параметров AlternatingLeastSquares не дало изменений в метрики: roc_auc_score = 0,9058.**

**Увеличение количества соседей в ItemItemRecommender до 10 увеличило roc_auc до 0,9499.**

**Применение взвешивания BM25 и TF-IDF и изменение их параметров к user_item_matrix снизило roc_auc до 0,9207.**

### Прогнозы

In [140]:
def get_predictions(targets_lvl_2, raw_predictions, prefix='lgb'): 
    df = targets_lvl_2[['user_id', 'item_id']]
    df['predictions'] = raw_predictions

    df = df.groupby(['user_id', 'item_id'])['predictions'].median().reset_index()
    df = df.sort_values(['predictions'], ascending=False).groupby(['user_id']).head(5)

    df = df.groupby('user_id')['item_id'].unique().reset_index()
    df.columns = ['user_id', prefix + '_recommendations']
    
    return df

In [141]:
def get_results(data_val_lvl_2, targets_lvl_2, preds_lgb, preds_cb, combined_preds):
    result = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
    result.columns=['user_id', 'actual']
    
    prefixes = ['lgb', 'cb', '2_mods']
    predictions = [preds_lgb, preds_cb, combined_preds]
    
    for i, preds in enumerate(predictions):
        df = get_predictions(targets_lvl_2, preds, prefixes[i])
        result = result.merge(df, on='user_id', how='left')

    return result

In [142]:
result_lvl_2 = get_results(data_val_lvl_2, targets_lvl_2, 
                           predictions_lgb_train,
                           predictions_cb_train, 
                           preds_train_2_mods)
result_lvl_2.head(2)

,user_id,actual,lgb_recommendations,cb_recommendations,2_mods_recommendations
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1082185, 865456, 8293439, 940947, 1087268]","[865456, 940947, 872137, 8293439, 1004906]","[865456, 1082185, 8293439, 940947, 9655212]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1082185, 899624, 965766, 5569230]","[1082185, 1106523, 899624, 910032, 883003]","[1106523, 1082185, 899624, 5569230, 883003]"


### Метрики качества

In [143]:
%%time
result_lvl_2.apply(lambda row: precision_at_k(row['cb_recommendations'], row['actual'], 5), axis=1).mean()

CPU times: total: 156 ms
Wall time: 143 ms


0.2622918707149854

In [144]:
%%time
result_lvl_2.apply(lambda row: precision_at_k(row['lgb_recommendations'], row['actual'], 5), axis=1).mean()

CPU times: total: 141 ms
Wall time: 135 ms


0.25132223310479923

In [145]:
%%time
result_lvl_2.apply(lambda row: precision_at_k(row['2_mods_recommendations'], row['actual'], 5), axis=1).mean()

CPU times: total: 141 ms
Wall time: 139 ms


0.260822722820764

### Прогнозы

In [146]:
validation_weeks = 6
data_train = data[data['week_no'] < data['week_no'].max() - validation_weeks]
data_valid = data[data['week_no'] >= data['week_no'].max() - validation_weeks]

In [147]:
users_lvl_1 = data_train.user_id.unique()
users_lvl_2 = data_valid.user_id.unique()
users_lvl_3 = test.user_id.unique()

new_users_lvl_2 = list(set(users_lvl_2) - set(users_lvl_1))
new_users_lvl_3 = list(set(users_lvl_3) - (set(users_lvl_1) | set(users_lvl_2)))

add_to_lvl_2 = list(set(users_lvl_3) - (set(users_lvl_2)))

new_users_lvl_2, new_users_lvl_3, len(add_to_lvl_2)

([], [2325], 75)

In [148]:
n_items_before = data['item_id'].nunique()
data_train = prefilter_items(data_train, item_features=item_features, take_n_popular=10000)
n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 89051 to 10001


In [149]:
user_item_features = get_new_features(data_train)
user_item_features.head(2)

,user_id,item_id,median_sales_hour,median_weekday,mean_visits_interval,mean_check,n_stores,n_items,n_transactions,mean_n_items_basket,max_n_items_basket,std_n_items_basket,mean_n_item_categories_basket,max_n_item_categories_basket,std_n_item_categories_basket
0,1,823721,13.0,4.0,NaN,37.772063,95,244,698,10.174603,25,6.32083,8.79661,20,4.837995
1,1,823990,15.0,6.0,NaN,37.772063,95,244,698,10.174603,25,6.32083,8.79661,20,4.837995


In [150]:
%%time
targets_test = get_targets_lvl_2(data_train, data_valid, user_item_features, N, add_to_lvl_2)

print(f'число пользователей: {targets_test.user_id.nunique()}')
print(f'среднее число покупок: {round(targets_test["target"].mean(), 4)}')

targets_test.head(2)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/10001 [00:00<?, ?it/s]

число пользователей: 2272
среднее число покупок: 0.0192
CPU times: total: 14min 48s
Wall time: 14min 44s


,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,manufacturer_freq,...,mean_check,n_stores,n_items,n_transactions,mean_n_items_basket,max_n_items_basket,std_n_items_basket,mean_n_item_categories_basket,max_n_item_categories_basket,std_n_item_categories_basket
0,84,999999,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.506667,493.0,86.0,163.0,3.52381,18.0,3.610541,3.1875,13.0,3.063179
1,84,1029743,1.0,69.0,GROCERY,0.0,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA,12676.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
X_train, X_valid, y_train, y_valid = train_test_split(targets_test[SELECTED_FEATURES_NAMES].fillna(0),
                                                      targets_test[['target']],
                                                      test_size=0.2, random_state=27,
                                                      stratify=targets_test[['target']])

dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)


model_lgb = lgb.train(params=params_lgb,
                      train_set=dtrain,
                      valid_sets=[dtrain, dvalid],
                      verbose_eval=500,
                      early_stopping_rounds=100) 

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 17551, number of negative: 896952
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.033516 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2812
[LightGBM] [Info] Number of data points in the train set: 914503, number of used features: 21
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.019192 -> initscore=-3.933891
[LightGBM] [Info] Start training from score -3.933891
Training until validation scores don't improve 

In [152]:
X_train, X_valid, y_train, y_valid = train_test_split(targets_test[SELECTED_FEATURES_NAMES].fillna(0),
                                                          targets_test[['target']],
                                                          test_size=0.2, random_state=27,
                                                          stratify=targets_test[['target']])
dtrain = Pool(data=X_train, label=y_train)
dvalid = Pool(data=X_valid, label=y_valid)


model_cb.fit(dtrain,
             eval_set=[dvalid]) 

0:	test: 0.8723376	best: 0.8723376 (0)	total: 229ms	remaining: 38m 13s
500:	test: 0.9292198	best: 0.9292198 (500)	total: 1m 47s	remaining: 33m 50s
1000:	test: 0.9323803	best: 0.9323803 (1000)	total: 3m 34s	remaining: 32m 4s
1500:	test: 0.9336892	best: 0.9336970 (1495)	total: 5m 21s	remaining: 30m 18s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9339984813
bestIteration = 1785

Shrink model to first 1786 iterations.


In [153]:
predictions_lgb_test = model_lgb.predict(targets_test[SELECTED_FEATURES_NAMES].fillna(0))
predictions_cb_test = model_cb.predict_proba(targets_test[SELECTED_FEATURES_NAMES].fillna(0))[:, 1]

In [154]:
preds_test_2_mods = predictions_lgb_test * 0.6 + predictions_cb_test * 0.4
roc_auc_score(targets_test['target'], preds_test_2_mods)

0.9509497356046952

In [155]:
result_test = get_results(test, targets_test, 
                           predictions_lgb_test,
                           predictions_cb_test, 
                           preds_test_2_mods)
result_test.head(2)

,user_id,actual,lgb_recommendations,cb_recommendations,2_mods_recommendations
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[1082185, 8293439, 9655212, 1131115, 865456]","[9655212, 8293439, 971585, 5577022, 1082185]","[8293439, 9655212, 1082185, 865456, 869465]"
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...","[1106523, 1075368, 1082185, 916122, 5569230]","[899624, 1106523, 1082185, 916122, 1075368]","[1106523, 899624, 1075368, 1082185, 916122]"


####  Метрики качества

In [156]:
%%time
result_test.apply(lambda row: precision_at_k(row['cb_recommendations'], row['actual'], 5), axis=1).mean()

CPU times: total: 125 ms
Wall time: 129 ms


0.23299734748010612

In [157]:
%%time
result_test.apply(lambda row: precision_at_k(row['lgb_recommendations'], row['actual'], 5), axis=1).mean()

CPU times: total: 125 ms
Wall time: 124 ms


0.23087533156498677

In [158]:
%%time
result_test.apply(lambda row: precision_at_k(row['2_mods_recommendations'], row['actual'], 5), axis=1).mean()

CPU times: total: 109 ms
Wall time: 121 ms


0.23660477453580905

In [159]:
df = result_test[['user_id', '2_mods_recommendations']].copy()
df.rename(columns = {'2_mods_recommendations':'actual'}, inplace = True)
df.to_csv('recommendations_for_users.csv', index=False)
df.head(2)

,user_id,actual
0,1,"[8293439, 9655212, 1082185, 865456, 869465]"
1,2,"[1106523, 899624, 1075368, 1082185, 916122]"


#### Вывод:
**Увеличение take_n_popular до 10000, увеличило roc_auc_score с 0,92 до 0,9509.
precision_at_k на тесте показало чуть меньше.**